# LaTeX Automation for NYCHA Waste Individual Action Plans

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from PIL import Image
import docx2txt
import re

In [2]:
#Set Global Vars and Options
os.chdir('/Users/kyleslugg/Documents/NYCHA/Production')
cons_tds = '073'
pd.set_option('display.max_columns', None)

## Parse and Process Text Blocks

TO COME:
- Waste Services and Assets
- Waste Distribution (top text -- bottom accounted for below)
- What Is an IAP?

In [ ]:
#Character Substitutions for LaTeX -- set and define "clean" method
def clean_text(text):
    substitutions = {'“':"``",
                '”': "''",
                '’':"'",
                ' ':' ',
                '–':'--',
                ' ':' ',
                '\xa0':' '}
    
    for key, value in substitutions.items():
        text = text.replace(key, value)
        
    return text

#### Overview Text

In [ ]:
def make_overview_text(cons_tds):
    header = re.compile(r'((\w*\s)*(Overview)):?')
    
    overview_text = docx2txt.process(f'TEXT/{cons_tds}_Overview.docx')
    try:
        overview_text = overview_text.replace(header.findall(overview_text)[0]+'\n','')
    except:
        overview_text = overview_text.replace(header.findall(overview_text)[0][0]+'\n','')
    overview_text = clean_text(overview_text)
    
    with open(f'TEXT/overview_text/{cons_tds}_overview.tex', 'w') as file_handle:
        file_handle.write(overview_text)
    
    pass

In [ ]:
make_overview_text(cons_tds)

#### Analysis Text

In [ ]:
def make_analysis_text(cons_tds):
    analysis_text = docx2txt.process(f'TEXT/{cons_tds}_Analysis.docx')

    header = re.compile(r'([\w\s]*:)')

    section_headings = {'Inspection and Collection Requirement':['Inspection and Collection Requirement',
                                                                'Inspection and Collection Requirements',
                                                                'Collection and Inspection Requirement',
                                                                'Collection and Inspection Requirements'],
                        
                        'Removal or Storage Requirement':['Removal or Storage Requirement',
                                                          'Removal or Storage Requirements',
                                                         'Removal and Storage Requirement',
                                                         'Removal and Storage Requirements',
                                                         'Storage or Removal Requirement',
                                                         'Storage and Removal Requirement',
                                                         'Storage and Removal Requirements']}
    
    for heading, variants in section_headings.items():
        for variant in variants:
            analysis_text = analysis_text.replace(variant, r'\emph{%s}' % heading)

    analysis_text = analysis_text.replace(header.findall(analysis_text)[0]+'\n','')

    latex_block = clean_text(analysis_text)

    with open(f'TEXT/analysis_text/{cons_tds}_analysis.tex', 'w') as file_handle:
        file_handle.write(latex_block)
        
    pass

In [ ]:
make_analysis_text(cons_tds)

## Select and Prepare Maps

In [ ]:
# Set asset map path
asset_map_path = f"MAPS/asset_maps/{cons_tds}_asset_map.png"

In [ ]:
# Split context map into two pages
def process_context_map(cons_tds):
    image = Image.open(f'REPORT_TEMPLATE/{cons_tds}_context_map.png')
    width, height = image.size

    bb1 = (0,0,width/2,height)
    bb2 = (width/2, 0, width, height)

    img_1 = image.crop(bb1)
    img_2 = image.crop(bb2)

    img_1.save(f'MAPS/context_maps/{cons_tds}_context_1.png', format="PNG")
    img_2.save(f'MAPS/context_maps/{cons_tds}_context_2.png', format="PNG")
    
    pass

## Produce Tables

TO COME:
- Waste Services and Assets
- Assets
- Capital Improvements

In [24]:
#Load Data
overview_data = pd.read_csv('DATA/overview_table_data.csv')
overview_data['CONS_TDS'] = overview_data['CONS_TDS'].apply(lambda x: str(x).zfill(3))
overview_data['TDS'] = overview_data['TDS'].apply(lambda x: str(x).zfill(3))

#### Make Overview Table

In [4]:
def make_overview_table(cons_tds, overview_data=overview_data):
    
    cons_data = overview_data.loc[overview_data['CONS_TDS']== cons_tds]
    
    overview_table = ''

    overview_frame = r'''
    \begin{tabular}{l|c|c|c|c|}
    \cline{2-5}
                                                                           & \cellcolor{ccteal}{\color[HTML]{FFFFFF} TDS \#} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Total Households} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Official Population} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Average Family Size} \\ \hline

    '''

    development_template = r'''\multicolumn{1}{|l|}{\cellcolor{ccteallight}%s}        & %s                                                   & %s                                                           & %s                                                                & %s                                                                \\ \hline'''


    overview_table += overview_frame

    for row in cons_data.itertuples():
        dev_name = row.DEV_NAME.title()
        dev_tds = row.TDS
        total_hhs = row.TOTAL_HH
        official_population = row.TOTAL_POP
        avg_family_size = row.AVG_FAMILY_SIZE

        overview_table += development_template % (dev_name, dev_tds, total_hhs, official_population, avg_family_size)

    overview_table += r'''
    \end{tabular}
    '''
    
    with open(f'TABLES/overview_table/{cons_tds}_overview_table.tex', 'w') as file_handle:
        file_handle.write(overview_table)
    

#### Waste Calculator

In [38]:
def add_waste_cols(overview_data=overview_data):
    conversion_factors = {'units_to_tons_day': 0.0025,
                         'cy_per_ton': {'trash': 21.05,
                                        'trash_actual': 0,
                                       'MGP': 18.02,
                                       'cardboard': 26.67,
                                       'paper': 6.19,
                                       'organics': 4.32,
                                       'ewaste': 5.65,
                                       'textiles': 13.33},
                         'gallons_per_cy': 201.974,
                         'gallons_per_64gal': 64,
                         'gallons_per_40lb_bag': 44,
                         'cy_per_44gal_bag':0.174,
                         'cy_per_cardboard_bale':0.193}

    waste_percentages = {'trash': .26,
                         'trash_actual':.894,
                        'MGP': .19,
                        'cardboard': .07,
                        'paper': .07,
                        'organics':.32,
                        'ewaste': .01,
                        'textiles': .08}

    capture_rates = {'trash_primary': .75,
                    'trash_secondary': .25,
                    'mgp': .30,
                    'cardboard': .50,
                    'paper': .20}

    overview_data['WASTE_TONS_DAY'] = overview_data['CURRENT_APTS'].apply(lambda x: x * conversion_factors['units_to_tons_day'])

    for key, value in waste_percentages.items():
        overview_data[f'{key.upper()}_CY'] = overview_data['WASTE_TONS_DAY'].apply(lambda x: x * value * conversion_factors['cy_per_ton'][key])
        overview_data[f'{key.upper()}_TONS'] = overview_data['WASTE_TONS_DAY'].apply(lambda x: x * value)
    
    overview_data['TRASH_ACTUAL_CY'] = (overview_data['TRASH_CY']+
                                           overview_data['MGP_CY']+
                                           overview_data['CARDBOARD_CY']+
                                           overview_data['PAPER_CY']+
                                           overview_data['ORGANICS_CY']+
                                           overview_data['EWASTE_CY']+
                                           overview_data['TEXTILES_CY'])-(overview_data['MGP_CY']*capture_rates['mgp']+
                                                                         overview_data['CARDBOARD_CY']*capture_rates['cardboard']+
                                                                         overview_data['PAPER_CY']*capture_rates['paper'])

    overview_data['TRASH_CHUTE_CY'] = overview_data['TRASH_ACTUAL_CY']*capture_rates['trash_primary']
    overview_data['TRASH_CHUTE_TONS'] = overview_data['TRASH_ACTUAL_TONS']*capture_rates['trash_primary']
    overview_data['TRASH_CHUTE_SAUSAGE'] = ((overview_data['TRASH_CHUTE_CY'])/conversion_factors['cy_per_ton']['trash'])*(2000/40)
    overview_data['TRASH_DROP_CY'] = overview_data['TRASH_ACTUAL_CY']*capture_rates['trash_secondary']
    overview_data['TRASH_DROP_TONS'] = overview_data['TRASH_ACTUAL_TONS']*capture_rates['trash_secondary']
    overview_data['TRASH_DROP_BINS'] = overview_data['TRASH_DROP_CY']*conversion_factors['gallons_per_cy']/64
    overview_data['CAPTURED_MGP_TONS_WEEK'] = overview_data['MGP_TONS']*capture_rates['mgp']*7
    overview_data['CAPTURED_CARDBOARD_TONS_WEEK'] = overview_data['CARDBOARD_TONS']*capture_rates['cardboard']*7
    overview_data['CAPTURED_PAPER_TONS_WEEK'] = overview_data['PAPER_TONS']*capture_rates['paper']*7
    overview_data['MGP_BAGS_WEEK'] = overview_data['MGP_CY']*capture_rates['mgp']*7/conversion_factors['cy_per_44gal_bag']
    overview_data['PAPER_BAGS_WEEK'] = overview_data['PAPER_CY']*capture_rates['paper']*7/conversion_factors['cy_per_44gal_bag']
    overview_data['CARDBOARD_BALES_WEEK'] = overview_data['CARDBOARD_CY']*capture_rates['cardboard']*7/conversion_factors['cy_per_cardboard_bale']
    
    return overview_data

In [39]:
add_waste_cols()

,GEO_BORO,NYCHA_BORO,CONS_TDS,CONS_NAME,DEV_NAME,TDS,AMP,TOTAL_HH,TOTAL_POP,AVG_FAMILY_SIZE,AVG_TENURE,CURRENT_APTS,TOTAL_APTS,STAIRHALLS,WASTE_TONS_DAY,TRASH_CY,TRASH_TONS,TRASH_ACTUAL_CY,TRASH_ACTUAL_TONS,MGP_CY,MGP_TONS,CARDBOARD_CY,CARDBOARD_TONS,PAPER_CY,PAPER_TONS,ORGANICS_CY,ORGANICS_TONS,EWASTE_CY,EWASTE_TONS,TEXTILES_CY,TEXTILES_TONS,TRASH_CHUTE_CY,TRASH_CHUTE_TONS,TRASH_CHUTE_SAUSAGE,TRASH_DROP_CY,TRASH_DROP_TONS,TRASH_DROP_BINS,CAPTURED_MGP_TONS_WEEK,CAPTURED_CARDBOARD_TONS_WEEK,CAPTURED_PAPER_TONS_WEEK,MGP_BAGS_WEEK,PAPER_BAGS_WEEK,CARDBOARD_BALES_WEEK
0,Bronx,Bronx,180,1010 EAST 178TH STREET,1010 EAST 178TH STREET,180,NY005011330,214,448,2.1,21.3,218,220,1.0,0.5450,2.982785,0.14170,6.352002,0.487230,1.865971,0.103550,1.017461,0.038150,0.236149,0.038150,0.753408,0.1744,0.030793,0.005450,0.581188,0.0436,4.764002,0.365423,11.315918,1.588001,0.121808,5.011482,0.217455,0.133525,0.053410,22.520340,1.900045,18.451356
1,Bronx,Bronx,308,Claremont Consolidated,1162-1176 WASHINGTON AVENUE,233,NY005013080,64,167,2.6,14.7,65,66,1.0,0.1625,0.889363,0.04225,1.893946,0.145275,0.556368,0.030875,0.303371,0.011375,0.070411,0.011375,0.224640,0.0520,0.009181,0.001625,0.173290,0.0130,1.420459,0.108956,3.374012,0.473486,0.036319,1.494249,0.064837,0.039813,0.015925,6.714780,0.566527,5.501551
2,Bronx,Bronx,067,Sotomayor,1471 WATSON AVENUE,214,NY005010670,96,158,1.6,23.2,96,96,1.0,0.2400,1.313520,0.06240,2.797212,0.214560,0.821712,0.045600,0.448056,0.016800,0.103992,0.016800,0.331776,0.0768,0.013560,0.002400,0.255936,0.0192,2.097909,0.160920,4.983157,0.699303,0.053640,2.206891,0.095760,0.058800,0.023520,9.917214,0.836717,8.125368
3,Bronx,Bronx,118,Adams,ADAMS,118,NY005001180,919,"2,253",2.5,22.5,924,925,7.0,2.3100,12.642630,0.60060,26.923166,2.065140,7.908978,0.438900,4.312539,0.161700,1.000923,0.161700,3.193344,0.7392,0.130515,0.023100,2.463384,0.1848,20.192374,1.548855,47.962884,6.730791,0.516285,21.241326,0.921690,0.565950,0.226380,95.453183,8.053403,78.206666
4,Bronx,Bronx,197,Fort Independence,BAILEY AVENUE-WEST 193RD STREET,202,NY005012020,233,459,2.0,20.0,232,233,1.0,0.5800,3.174340,0.15080,6.759929,0.518520,1.985804,0.110200,1.082802,0.040600,0.251314,0.040600,0.801792,0.1856,0.032770,0.005800,0.618512,0.0464,5.069947,0.388890,12.042629,1.689982,0.129630,5.333320,0.231420,0.142100,0.056840,23.966600,2.022067,19.636306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Staten Island,Queens-Staten Island,035,South Beach,SOUTH BEACH,035,NY005010350,411,881,2.1,23.2,421,422,15.0,1.0525,5.760333,0.27365,12.266940,0.940935,3.603550,0.199975,1.964912,0.073675,0.456048,0.073675,1.454976,0.3368,0.059466,0.010525,1.122386,0.0842,9.200205,0.705701,21.853219,3.066735,0.235234,9.678137,0.419948,0.257862,0.103145,43.491115,3.669354,35.633124
298,Staten Island,Mixed Finance,114,Stapleton,STAPLETON,114,NY005021140,638,"1,974",3.1,16.2,693,693,12.0,1.7325,9.481973,0.45045,20.192374,1.548855,5.931733,0.329175,3.234404,0.121275,0.750692,0.121275,2.395008,0.5544,0.097886,0.017325,1.847538,0.1386,15.144281,1.161641,35.972163,5.048094,0.387214,15.930994,0.691267,0.424463,0.169785,71.589887,6.040053,58.654999
299,Staten Island,Queens-Staten Island,042,Todt Hill,TODT HILL,042,NY005000520,490,985,2.0,22.5,502,502,14.0,1.2550,6.868615,0.32630,14.627088,1.121970,4.296869,0.238450,2.342960,0.087850,0.543792,0.087850,1.734912,0.4016,0.070908,0.012550,1.338332,0.1004,10.970316,0.841478,26.057757,3.656772,0.280493,11.540201,0.500745,0.307475,0.122990,51.858764,4.375334,42.488903
300,Staten Island,Queens-Staten Island,116,West Brighton,WEST BRIGHTON I,116,NY005010130,475,"1,324",2.8,17.9,487,490,9.0,1.2175,6.663378,0.31655,14.190023,1.088445,4.168476,0.231325,2.272951,0.085225,0.527543,0.085225,1.683072,0.3896,0.068789,0.012175,1.298342,0.0974,10.642518,0.816334,25.279139,3.547506,0.272111,11.195374,0.485783,0.298288,0.119315,50.309199,

In [40]:
def make_waste_distribution_table(cons_tds, overview_data=overview_data):
    cons_data = overview_data[overview_data['CONS_TDS'] == cons_tds]
    num_devs = cons_data.shape[0]
    
    if num_devs > 1:
        num_cols = num_devs+1
    else:
        num_cols = num_devs

    dev_col_format = r'X|'

    opening = r'''
    \begin{tabularx}{\textwidth}{V{1.5in}|%s}
    \cline{2-%s}
    ''' % (dev_col_format*num_cols, (num_cols+1))

    top_row = r'''
                                                                   '''+(r"& \multicolumn{1}{l|}{\cellcolor{ccorange}%s}"*(num_cols))+r"\tnhl"+'\n'

    standard_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

    captured_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Captured / Week (tons)\tnote{4}}                        "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

    chute_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Trash Chutes\tnote{2}}                 "+(r"& %s tons or (%s) 40 lbs. sausage bags      "*num_cols)+r"\tnhl"+'\n'

    dropsite_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Drop Sites\tnote{3}}                 "+(r"& %s tons or (%s) 64-gallon bins      "*num_cols)+r"\tnhl"+'\n'

    OET_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s / Day (CY)\tnote{5}}              "+(r"& %s                                    "*num_cols)+r"\tnhl"+'\n'
    
    recycling_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s tons or (%s) 44-gallon bags                                   ")*num_cols+r"\tnhl"+'\n'
    
    cardboard_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s tons or (%s) bales                                   ")*num_cols+r"\tnhl"+'\n'
    
    def make_waste_distribution_table_block(cons_data, num_cols):
        if num_cols != 1:
            cons_data.loc['Total']= cons_data.sum(numeric_only=True, axis=0)
            cons_data.loc['Total','DEV_NAME'] = 'Total'

        def make_trash_text(row, text_var, cy_col, other_col):
            text_var.append(round(row[cy_col],2))
            text_var.append(round(row[other_col], 2))
            pass

        latex_block = opening
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
        latex_block += standard_row % tuple([r"Waste Generated / Day (Tons)\tnote{1}"]+[round(item, 2) for item in cons_data['WASTE_TONS_DAY'].tolist()])
        latex_block += standard_row % tuple([r"Trash / Day (tons)\tnote{2}"]+cons_data['TRASH_ACTUAL_TONS'].apply(lambda x: str(round(x,2))).tolist())

        trash_chute_text = []
        dropsite_text = []

        cons_data.apply(lambda row: make_trash_text(row, trash_chute_text, 'TRASH_CHUTE_TONS', 'TRASH_CHUTE_SAUSAGE'), axis=1)
        cons_data.apply(lambda row: make_trash_text(row, dropsite_text, 'TRASH_DROP_TONS', 'TRASH_DROP_BINS'), axis=1)

        latex_block += chute_row % tuple(trash_chute_text)
        latex_block += dropsite_row % tuple(dropsite_text)
        
        latex_block += r"\end{tabularx}\bigskip"
        
        latex_block += opening
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
        
        mgp_text = []
        cardboard_text= []
        paper_text = []
        
        cons_data.apply(lambda row: make_trash_text(row, mgp_text, 'CAPTURED_MGP_TONS_WEEK', 'MGP_BAGS_WEEK'), axis=1)

        cons_data.apply(lambda row: make_trash_text(row, cardboard_text, 'CAPTURED_CARDBOARD_TONS_WEEK', 'CARDBOARD_BALES_WEEK'), axis=1)

        cons_data.apply(lambda row: make_trash_text(row, paper_text, 'CAPTURED_PAPER_TONS_WEEK', 'PAPER_BAGS_WEEK'), axis=1)


        
        latex_block += recycling_row % tuple([r"Metal, Glass, Plastic Captured / Week (tons)"]+mgp_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_MGP_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += cardboard_row % tuple([r"Cardboard Captured / Week (tons)"]+cardboard_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_CARDBOARD_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += recycling_row % tuple([r"Paper Captured / Week (tons)"]+paper_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_PAPER_CY'].apply(lambda x: str(round(x,2))).tolist())

        #latex_block += OET_row % tuple(['Organics']+cons_data['ORGANICS_CY'].apply(lambda x: str(round(x,2))).tolist())
        #latex_block += OET_row % tuple(['E-Waste']+cons_data['EWASTE_CY'].apply(lambda x: str(round(x,2))).tolist())
        #latex_block += OET_row % tuple(['Textiles']+cons_data['TEXTILES_CY'].apply(lambda x: str(round(x,2))).tolist())

        latex_block += r"\end{tabularx}"

        return latex_block
    
    
    latex_block = make_waste_distribution_table_block(cons_data, num_cols)
    
    with open(f'TABLES/waste_distribution_table/{cons_tds}_wd_table.tex', 'w') as file_handle:
        file_handle.write(latex_block)
    
    '''
    text_block = r''''''
    
    def make_text_line(row, text_block=text_block):
    
        text_line_multi = r"\bf{%s}: This development has %s apartment units and %s stairhalls.\\"

        text_line_singular = r"\bf{%s}: This development has %s apartment units and one stairhall.\\"
        
        if int(row['STAIRHALLS']) == 1:
            text_block += text_line_singular % (row['DEV_NAME'].title(), row['CURRENT_APTS'])
        else:
            text_block += text_line_multi % (row['DEV_NAME'].title(), row['CURRENT_APTS'], int(row['STAIRHALLS']))
        
        pass
    
    cons_data.apply(lambda row: make_text_line(row), axis=1)

    
    with open(f'TEXT/waste_distribution_bottom/{cons_tds}_wd_bottom.tex', 'w') as file_handle:
        file_handle.write(text_block)
    '''
    

In [41]:
make_waste_distribution_table('073')

/Users/kyleslugg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/kyleslugg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


#### Make Capital Improvements Table

In [44]:
asset_data = {'fwd': ['In-Sink Food Grinders', pd.read_csv('DATA/capital_fwd.csv')],
              'ehd': ['Enlarged Hopper Doors', pd.read_csv('DATA/capital_ehd.csv')],
              'int_compactor':['Interior Compactor Replacement', pd.read_csv('DATA/capital_intcom.csv')],
              'wasteyard':['Waste Yard Redesign', pd.read_csv('DATA/capital_wasteyard.csv')]}

In [58]:
cons_data = overview_data[overview_data['CONS_TDS'] == cons_tds]
num_devs = cons_data.shape[0]

dev_col_format = r'X|'
header = r'''
\begin{tabularx}{\textwidth}{r|%s}
\cline{2-%s}
''' % ((dev_col_format*num_devs), num_devs)

top_row = r"\multicolumn{1}{l|}{}                                                        "+r"& \cellcolor{ccorange}{\color[HTML]{FFFFFF}%s} )"*num_devs+r"\\ \hline"+"\n"

project_block = r"\multicolumn{1}{|V{.2\columnwidth}|}{\cellcolor{ccorangelight}%s}          "+(r"&                                                                  "*num_devs)+r"\\"+r'''
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Status}}                '''+(r"& %s                                                         "*num_devs)+r'''\\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{%s}}                  '''+("& %s                                                     "*num_devs)+r"\\ \hline"+"\n"

In [62]:
#NOT WORKING YET!

def make_capital_table_block(cons_data):
    devs = cons_data['DEV_NAME'].apply(lambda x: str(x).upper()).tolist()
    devs_title = cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist()
    latex_block = ''
    latex_block += header
    latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
    
    for asset in asset_data.keys():
        asset_df = asset_data[asset][1]
        #print(asset_data[asset][0])
        #print(devs)
        #print(asset_df['DEVELOPMENT'].tolist())
        if any((dev in asset_df['DEVELOPMENT'].tolist()) for dev in devs):
            status_list = []
            cost_title = []
            cost_list = []
            
            for dev in devs:
                if dev in asset_df['DEVELOPMENT']:
                    status_list.append(asset_df.loc(asset_df['DEVELOPMENT']==dev, 'STATUS'))
                    
                    if (asset_df.loc(asset_df['DEVELOPMENT']== dev, 'COST')== '$-') or (pd.isna(asset_df.loc(asset_df['DEVELOPMENT']==dev, 'COST'))):
                        if (asset_df.loc(asset_df['DEVELOPMENT']== dev, 'ESTIMATE')== '$-' or pd.isna(asset_df.loc(asset_df['DEVELOPMENT']==dev, 'ESTIMATE'))):
                            cost_list.append(' ')
                        else:
                            cost_list.append('\\'+str(asset_df.loc(asset_df['DEVELOPMENT']==dev), 'ESTIMATE')+r" (est.)")
                    else:
                        if (asset_df.loc(asset_df['DEVELOPMENT']==dev, 'COST')== '$-' or pd.isna(asset_df.loc(asset_df['DEVELOPMENT']==dev, 'COST'))):
                            cost_list.append(' ')                    
                        else: 
                            cost_list.append('\\'+asset_df.loc(asset_df['DEVELOPMENT']==dev, 'COST'))
                    
                else:
                    status_list.append('N/A')
                    cost_list.append(' ')
                
            if all(item == ' ' for item in cost_list):
                cost_title.append(' ')
            else:
                cost_title.append('Cost')
            
            asset_block = project_block % tuple([asset_data[asset][0]]+status_list+cost_title+cost_list)
            
            latex_block += asset_block
            
            latex_block += r"\end{tabularx}"
        
    return latex_block
        
                
                    
        

In [61]:
with open(f"TABLES/capital_projects_table/{cons_tds}_capital_projects.tex", 'w') as file_handle:
    file_handle.write(make_capital_table_block(cons_data))

In [ ]:
'''
\begin{tabularx}{\textwidth}{r|X|X|X|}
\cline{2-4}
\multicolumn{1}{l|}{}                                                        & \cellcolor{ccorange}{\color[HTML]{FFFFFF} 303 Vernon Avenue} & \cellcolor{ccorange}{\color[HTML]{FFFFFF} Bedford-Stuyvesant Rehab} & \cellcolor{ccorange}{\color[HTML]{FFFFFF} Sumner Houses} \\ \hline
\multicolumn{1}{|V{.2\columnwidth}|}{\cellcolor{ccorangelight}In-Sink Food Grinders}          &                                                                  &                                                                         &                                                              \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Status}}                & Estimate                                                         & Estimate                                                                & Estimate                                                     \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Cost}}                  & \$402,184.21                                                     & \$144,373.82                                                            & \$1,887,172.07                                               \\ \hline
\multicolumn{1}{|V{.2\columnwidth}|}{\cellcolor{ccorangelight}Enlarged Hopper Doors}          &                                                                  &                                                                         &                                                              \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Status}}                & Estimate                                                         & Queued                                                                  & Completed                                                    \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Cost}}                  & \$7,435.64                                                       & \$37,178.22                                                             & \$148,620.14                                                 \\ \hline
\multicolumn{1}{|V{.2\columnwidth}|}{\cellcolor{ccorangelight}Interior Compactor Replacement} &                                                                  &                                                                         &                                                              \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Status}}                & Queued                                                           & In Progress                                                             & Completed                                                    \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Cost}}                  & \$63,087.50                                                      & \$247,785.00                                                            & \$957,660.00                                                 \\ \hline
\multicolumn{1}{|V{.2\columnwidth}|}{\cellcolor{ccorangelight}Waste Yard Redesign}            &                                                                  &                                                                         &                                                              \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Status}}                & Estimate                                                         & Estimate                                                                & Estimate                                                     \\
\multicolumn{1}{|r|}{\cellcolor{ccorangelight}}                               &                                                                  &                                                                         &                                                              \\ \hline
\end{tabularx}
''''''

#### Make Staff Table

In [ ]:
cons_ids = pd.read_csv('DATA/CONS_NAME_TDS.csv')
cons_ids['CONS_TDS'] = cons_ids['CONS_TDS'].apply(lambda x: str(x).zfill(3))

dev_staff = pd.read_csv('DATA/staff_for_table.csv')
dev_staff.dropna(inplace=True)
dev_staff['Consolidation'] = dev_staff['Consolidation'].apply(lambda x: str(x).upper())
#Note: Staff list missing for Armstrong, Ft. Washington, and Williams Plaza, as well as scatter-site third-party-managed consolidations
dev_staff = dev_staff.merge(cons_ids, left_on='Consolidation', right_on='CONS_NAME', how='inner', indicator=False)

In [ ]:
table_frame = pd.read_csv('DATA/Table_Keys.csv')

In [ ]:
def make_staff_table(cons_tds):
    cons_id = cons_ids['CONS_NAME'].loc[cons_ids['CONS_TDS'] == cons_tds].iloc[0]
    
    #Fetching staff data for consolidation
    cons_data = dev_staff.loc[dev_staff['Consolidation'] == cons_id]
    
    #Setting up table and transposing data
    cons_table_frame = table_frame
    cons_table_frame['Budgeted'] = cons_table_frame['BUDG_KEY'].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame['Formula'] = cons_table_frame['FORMULA_KEY'].iloc[:-1].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame.loc[cons_table_frame['CHART_LINE']== 'Caretakers J', 'Variance'] = (cons_table_frame.iloc[-1]['Budgeted']+
                                             cons_table_frame.iloc[-2]['Budgeted']-
                                             cons_table_frame.iloc[-2]['Formula'])
    
    #Simplifying table
    cons_table = cons_table_frame[['CHART_LINE', 'Budgeted', 'Formula', 'Variance']]
    
    #Defining LaTeX table format
    
    def make_staff_table_block(staff_data):
    
        table_template = r'''
        \begin{tabular}{l|c|c|c|}
        \cline{2-4}
                                                                                     & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Budgeted} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Formula Allocation} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Variance} \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Employees}                      & %s                                                      & %s                                                                & %s                                                        \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Property Manager}               & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Asst. Property Manager}         & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Secretaries}                    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Housing Assistants}             & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Superintendent}                 & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Assistant Superintendent}       & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Caretakers (SOC)} & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Grounds (SOG)}    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Maintenance Workers}            & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers X}                   & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers J\tnote{1}}                   & %s                                                      &                                                                 &                                                         \\ \cline{1-2}
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers G}                   & %s                                                      & \multirow{-2}{*}{%s}                                     & \multirow{-2}{*}{%s}                           \\ \hline
        \end{tabular}
        
        '''

        values = []

        def extract_data_through_ctx(row):
            [values.append(item) for item in [str(int(row['Budgeted'])), 
                                              str(int(row['Formula'])), 
                                              str(int(row['Variance']))]]
            pass

        #Processing through Caretaker X
        staff_data.iloc[0:-2].apply(lambda row: extract_data_through_ctx(row), axis=1)

        #Processing Caretaker J and G
        values.append(str(int(staff_data.iloc[-2, 1])))
        values.append(str(int(staff_data.iloc[-1, 1])))
        values.append(str(int(staff_data.iloc[-2, 2])))
        values.append(str(int(staff_data.iloc[-2, 3])))

        return table_template % tuple(values)
    
    #Make and export LaTeX code
    with open(f'TABLES/staff_table/{cons_tds}_staff_table.tex', 'w') as file_handle:
        file_handle.write(make_staff_table_block(cons_table))